In [ ]:
import os
import cv2
import csv
import PIL
import numpy as np
import pandas as pd
from PIL import Image
import mediapipe as mp

In [ ]:
drawingModule = mp.solutions.drawing_utils
handsModule = mp.solutions.hands
mp_model = handsModule.Hands(static_image_mode = True)

In [ ]:
def input_img(filePath, fileName):
    
    load_path = filePath + fileName
    save_path = filePath + fileName
    
    return load_path, save_path

In [ ]:
def mediapipe_detection(image, fileName, mp_model):
    
    results = mp_model.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    
    if results.multi_hand_landmarks != None:
        for handLandmarks in results.multi_hand_landmarks:
            drawingModule.draw_landmarks(image, handLandmarks, handsModule.HAND_CONNECTIONS, 
                                         drawingModule.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4), 
                                         drawingModule.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2))
    
    img = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    
    return img, results

In [ ]:
def hand_detection(results):
    
    hand = 'none'

    if len(results.multi_handedness) == 1:
        if results.multi_handedness[0].classification[0].label == 'Left':
            hand = 'left'
        if results.multi_handedness[0].classification[0].label == 'Right':
            hand = 'right'
    else:
        hand = 'both'

    return hand

In [ ]:
def extract_keypoints(results):
    
    kp_list = []
    
    for data_point in results.multi_hand_landmarks[0].landmark:
        kp_list.append([data_point.x, data_point.y, data_point.z])
    
    keypoints = np.array(kp_list).flatten()
    
    return keypoints

In [ ]:
def extract_keypoints_both(hand, results):
    
    kp_list = []
    
    if hand == 'right':
        for i in range(0, 21):
            kp_list.append([0, 0, 0])
        
        for data_point in results.multi_hand_landmarks[0].landmark:
            kp_list.append([data_point.x, data_point.y, data_point.z])
        
    if hand == 'left':
        for data_point in results.multi_hand_landmarks[0].landmark:
            kp_list.append([data_point.x, data_point.y, data_point.z])
        
        for i in range(0, 21):
            kp_list.append([0, 0, 0])
    
    if hand == 'both':
        for data_point in results.multi_hand_landmarks[0].landmark:
            kp_list.append([data_point.x, data_point.y, data_point.z])
        
        for data_point in results.multi_hand_landmarks[1].landmark:
            kp_list.append([data_point.x, data_point.y, data_point.z])
    
    keypoints = np.array(kp_list).flatten()
    
    return keypoints

In [ ]:
def create_csv(csv_file):
    
    columns = ['letter']
    
    for val in range(1, 22):
        columns += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val)]
    
    with open(csv_file, mode='w', newline='') as f:
        csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerow(columns)

In [ ]:
def write_csv(letter, keypoints, csv_file):
    
    temp = list(keypoints)
    temp.insert(0, letter)
    
    with open(csv_file, mode='a', newline='') as f:
        csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerow(temp)

In [ ]:
directory = 'MediaPipe'
csv_file = 'features.csv'
create_csv(csv_file)
os.chdir(directory)
list_dir = os.listdir()

for label in list_dir:
    os.chdir(label)
    images = os.listdir()
    os.chdir("..")
    os.chdir("..")
    for i in images:
        load_path, save_path = input_img(directory + '/' + label + '/', i)
        img, results = mediapipe_detection(load_path, mp_model)
        try:
            keypoints = extract_keypoints(results)
            write_csv(label, keypoints, csv_file)
            np.save(save_path, keypoints)
        except:
    
    os.chdir(directory)

os.chdir("..")